# 🔬 LLM SAE Stability Analysis - Production Grade

**Research Question:** Does SAE stability decrease monotonically with sparsity on LLMs, as we found on algorithmic tasks?

## Key Findings from Algorithmic Tasks
- Stability DECREASES monotonically with L0 (sparsity)
- This is architecture-independent (TopK, ReLU, Gated, JumpReLU)
- Correlation: -0.725 (strong negative)

## This Notebook Tests
1. **Pretrained SAE Analysis**: Compare Gemma Scope SAEs at different widths
2. **Multi-Seed Training**: Train SAEs with same config, different seeds
3. **Stability vs Sparsity**: Vary L1 coefficient and measure PWMCC

## Setup
1. **Enable GPU**: Runtime > Change runtime type > GPU (T4 is sufficient)
2. **Run all cells** in order
3. **Expected runtime**: ~30-60 minutes for full analysis

In [ ]:
#@title 1. Install Dependencies
!pip install sae-lens transformer-lens transformers datasets -q
!pip install einops jaxtyping -q

In [ ]:
#@title 2. Imports and GPU Check
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import gc
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')
if device == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️ No GPU detected! Enable GPU: Runtime > Change runtime type > GPU')

In [ ]:
#@title 3. Utility Functions

def compute_pwmcc(d1: torch.Tensor, d2: torch.Tensor) -> float:
    """Compute Pairwise Maximum Cosine Correlation between decoder matrices.
    
    Args:
        d1: [d_sae, d_model] decoder matrix
        d2: [d_sae, d_model] decoder matrix
    
    Returns:
        PWMCC in [0, 1]
    """
    d1_norm = F.normalize(d1, dim=1)
    d2_norm = F.normalize(d2, dim=1)
    cos_sim = d1_norm @ d2_norm.T
    max_1to2 = cos_sim.abs().max(dim=1)[0].mean().item()
    max_2to1 = cos_sim.abs().max(dim=0)[0].mean().item()
    return (max_1to2 + max_2to1) / 2


def compute_random_baseline(d_model: int, d_sae: int, n_trials: int = 10) -> float:
    """Compute expected PWMCC for random matrices."""
    pwmcc_values = []
    for _ in range(n_trials):
        d1 = torch.randn(d_sae, d_model, device=device)
        d2 = torch.randn(d_sae, d_model, device=device)
        pwmcc_values.append(compute_pwmcc(d1, d2))
    return np.mean(pwmcc_values)


def compute_feature_overlap_stats(d1: torch.Tensor, d2: torch.Tensor) -> Dict:
    """Compute detailed feature overlap statistics."""
    d1_norm = F.normalize(d1, dim=1)
    d2_norm = F.normalize(d2, dim=1)
    cos_sim = d1_norm @ d2_norm.T
    
    max_sim = cos_sim.abs().max(dim=1)[0]
    
    return {
        'mean_max_sim': max_sim.mean().item(),
        'std_max_sim': max_sim.std().item(),
        'pct_above_90': (max_sim > 0.9).float().mean().item() * 100,
        'pct_above_80': (max_sim > 0.8).float().mean().item() * 100,
        'pct_above_70': (max_sim > 0.7).float().mean().item() * 100,
        'pct_above_50': (max_sim > 0.5).float().mean().item() * 100,
    }


def clear_gpu_memory():
    """Clear GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


print('✓ Utility functions loaded')

---
# Part 1: Analyze Pretrained SAEs (Gemma Scope)

We'll load pretrained SAEs from Gemma Scope and analyze:
1. Feature overlap across different widths (dictionary sizes)
2. Feature overlap across different layers

In [ ]:
#@title 4. Load Gemma Scope SAEs
from sae_lens import SAE

# Configuration
MODEL = 'gemma-2-2b'  # Options: 'gemma-2-2b', 'gemma-2-9b'
LAYER = 12  # Middle layer
WIDTHS = ['16k', '32k', '65k']  # Different dictionary sizes

print(f'Loading SAEs for {MODEL}, layer {LAYER}')
print('=' * 50)

saes = {}
for width in WIDTHS:
    print(f'\nLoading width {width}...')
    try:
        sae_id = f'layer_{LAYER}/width_{width}/canonical'
        sae, cfg, sparsity = SAE.from_pretrained(
            release='gemma-scope-2b-pt-res-canonical',
            sae_id=sae_id,
            device=device
        )
        saes[width] = {
            'sae': sae,
            'd_sae': sae.cfg.d_sae,
            'd_model': sae.cfg.d_in,
            'sparsity': sparsity
        }
        print(f'  ✓ d_sae={sae.cfg.d_sae:,}, d_model={sae.cfg.d_in}')
        if sparsity:
            print(f'  ✓ Reported L0: {sparsity}')
    except Exception as e:
        print(f'  ✗ Error: {e}')

print(f'\n✓ Loaded {len(saes)} SAEs')

In [ ]:
#@title 5. Analyze Feature Overlap Across Widths
print('Feature Overlap Analysis Across Widths')
print('=' * 60)

width_results = []
width_list = list(saes.keys())

for i, w1 in enumerate(width_list):
    for w2 in width_list[i+1:]:
        d1 = saes[w1]['sae'].W_dec.data
        d2 = saes[w2]['sae'].W_dec.data
        
        stats = compute_feature_overlap_stats(d1, d2)
        
        print(f'\n{w1} ({saes[w1]["d_sae"]:,}) → {w2} ({saes[w2]["d_sae"]:,}):')
        print(f'  Mean max cosine sim: {stats["mean_max_sim"]:.3f} ± {stats["std_max_sim"]:.3f}')
        print(f'  Features with >0.9 match: {stats["pct_above_90"]:.1f}%')
        print(f'  Features with >0.8 match: {stats["pct_above_80"]:.1f}%')
        print(f'  Features with >0.7 match: {stats["pct_above_70"]:.1f}%')
        
        width_results.append({
            'width1': w1,
            'width2': w2,
            **stats
        })

In [ ]:
#@title 6. Analyze Cross-Layer Feature Overlap
LAYERS = [6, 12, 18, 24]  # Early, middle, late layers
WIDTH = '16k'  # Use smallest for memory efficiency

print(f'Loading SAEs across layers: {LAYERS}')
print('=' * 50)

layer_saes = {}
for layer in LAYERS:
    print(f'Loading layer {layer}...')
    try:
        sae_id = f'layer_{layer}/width_{WIDTH}/canonical'
        sae, cfg, sparsity = SAE.from_pretrained(
            release='gemma-scope-2b-pt-res-canonical',
            sae_id=sae_id,
            device=device
        )
        layer_saes[layer] = sae
        print(f'  ✓ Loaded')
    except Exception as e:
        print(f'  ✗ Error: {e}')

# Compute cross-layer PWMCC
print('\nCross-Layer Feature Similarity')
print('=' * 50)

layer_list = sorted(layer_saes.keys())
d_model = layer_saes[layer_list[0]].cfg.d_in
d_sae = layer_saes[layer_list[0]].cfg.d_sae
random_baseline = compute_random_baseline(d_model, d_sae)
print(f'Random baseline: {random_baseline:.4f}')

layer_results = []
for i in range(len(layer_list)):
    for j in range(i + 1, len(layer_list)):
        l1, l2 = layer_list[i], layer_list[j]
        d1 = layer_saes[l1].W_dec.data
        d2 = layer_saes[l2].W_dec.data
        
        pwmcc = compute_pwmcc(d1, d2)
        ratio = pwmcc / random_baseline
        
        print(f'\nLayer {l1} vs {l2}:')
        print(f'  PWMCC: {pwmcc:.4f} ({ratio:.2f}× random)')
        
        layer_results.append({
            'layer1': l1,
            'layer2': l2,
            'pwmcc': pwmcc,
            'ratio': ratio
        })

---
# Part 2: Train SAEs with Different Seeds

This is the key experiment: train multiple SAEs with the same configuration but different random seeds, then measure PWMCC between them.

In [ ]:
#@title 7. Setup SAE Training
from transformer_lens import HookedTransformer
from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner

# Clear memory from previous SAEs
del saes, layer_saes
clear_gpu_memory()

# Load model
print('Loading Gemma 2 2B...')
model = HookedTransformer.from_pretrained(
    'gemma-2-2b',
    device=device,
    dtype=torch.float16  # Use FP16 for memory efficiency
)
print(f'✓ Model loaded: {model.cfg.n_layers} layers, d_model={model.cfg.d_model}')

In [ ]:
#@title 8. Train Multiple SAEs with Different Seeds

# Training configuration
LAYER = 12
EXPANSION_FACTOR = 4  # d_sae = 4 * d_model (smaller for faster training)
L1_COEFFICIENT = 5e-3  # Controls sparsity
TOTAL_TOKENS = 500_000  # Small for demo (increase for production)
N_SEEDS = 3  # Number of seeds to train

print(f'Training {N_SEEDS} SAEs with different seeds')
print(f'Layer: {LAYER}, Expansion: {EXPANSION_FACTOR}x, L1: {L1_COEFFICIENT}')
print(f'Total tokens: {TOTAL_TOKENS:,}')
print('=' * 60)

trained_saes = []
seeds = [42, 123, 456]

for i, seed in enumerate(seeds[:N_SEEDS]):
    print(f'\n[{i+1}/{N_SEEDS}] Training with seed {seed}...')
    
    cfg = LanguageModelSAERunnerConfig(
        model_name='gemma-2-2b',
        hook_point=f'blocks.{LAYER}.hook_resid_post',
        hook_point_layer=LAYER,
        d_in=model.cfg.d_model,
        dataset_path='monology/pile-uncopyrighted',
        streaming=True,
        
        # SAE architecture
        expansion_factor=EXPANSION_FACTOR,
        b_dec_init_method='zeros',
        
        # Training
        lr=3e-4,
        l1_coefficient=L1_COEFFICIENT,
        train_batch_size_tokens=4096,
        context_size=128,
        seed=seed,
        
        # Data loading
        n_batches_in_buffer=32,
        total_training_tokens=TOTAL_TOKENS,
        store_batch_size_prompts=16,
        
        # Logging
        log_to_wandb=False,
    )
    
    runner = SAETrainingRunner(cfg)
    sae = runner.run()
    trained_saes.append({'seed': seed, 'sae': sae})
    
    print(f'  ✓ Trained SAE with seed {seed}')
    clear_gpu_memory()

print(f'\n✓ Trained {len(trained_saes)} SAEs')

In [ ]:
#@title 9. Compute Stability (PWMCC) Between Trained SAEs
print('Computing PWMCC between trained SAEs')
print('=' * 50)

# Get decoder matrices
decoders = []
for item in trained_saes:
    decoder = item['sae'].W_dec.data.clone()
    decoders.append(decoder)

# Compute random baseline
d_sae, d_model = decoders[0].shape
random_baseline = compute_random_baseline(d_model, d_sae)
print(f'Random baseline: {random_baseline:.4f}')

# Compute pairwise PWMCC
pwmcc_values = []
for i in range(len(decoders)):
    for j in range(i + 1, len(decoders)):
        pwmcc = compute_pwmcc(decoders[i], decoders[j])
        ratio = pwmcc / random_baseline
        pwmcc_values.append(pwmcc)
        
        print(f'\nSeed {trained_saes[i]["seed"]} vs {trained_saes[j]["seed"]}:')
        print(f'  PWMCC: {pwmcc:.4f} ({ratio:.2f}× random)')

# Summary
mean_pwmcc = np.mean(pwmcc_values)
std_pwmcc = np.std(pwmcc_values)
mean_ratio = mean_pwmcc / random_baseline

print('\n' + '=' * 50)
print('SUMMARY')
print('=' * 50)
print(f'Mean PWMCC: {mean_pwmcc:.4f} ± {std_pwmcc:.4f}')
print(f'Ratio to random: {mean_ratio:.2f}×')

if mean_ratio < 1.2:
    print('\n⚠️ Stability is near random baseline!')
elif mean_ratio < 1.5:
    print('\n📊 Moderate stability above random')
else:
    print('\n✓ Good stability above random')

---
# Part 3: Stability vs Sparsity Analysis

Train SAEs at different sparsity levels (L1 coefficients) and measure stability at each level.

In [ ]:
#@title 10. Stability vs Sparsity Experiment

# Clear previous SAEs
del trained_saes, decoders
clear_gpu_memory()

# L1 coefficients to test (higher = sparser)
L1_COEFFICIENTS = [1e-3, 3e-3, 5e-3, 1e-2]
N_SEEDS_PER_L1 = 2  # Train 2 SAEs per L1 to measure stability
TOTAL_TOKENS = 300_000  # Smaller for speed

print('Stability vs Sparsity Experiment')
print(f'L1 coefficients: {L1_COEFFICIENTS}')
print(f'Seeds per L1: {N_SEEDS_PER_L1}')
print('=' * 60)

stability_results = []

for l1_coef in L1_COEFFICIENTS:
    print(f'\n--- L1 = {l1_coef} ---')
    
    saes_at_l1 = []
    for seed in [42, 123][:N_SEEDS_PER_L1]:
        print(f'  Training seed {seed}...')
        
        cfg = LanguageModelSAERunnerConfig(
            model_name='gemma-2-2b',
            hook_point=f'blocks.{LAYER}.hook_resid_post',
            hook_point_layer=LAYER,
            d_in=model.cfg.d_model,
            dataset_path='monology/pile-uncopyrighted',
            streaming=True,
            expansion_factor=4,
            b_dec_init_method='zeros',
            lr=3e-4,
            l1_coefficient=l1_coef,
            train_batch_size_tokens=4096,
            context_size=128,
            seed=seed,
            n_batches_in_buffer=32,
            total_training_tokens=TOTAL_TOKENS,
            store_batch_size_prompts=16,
            log_to_wandb=False,
        )
        
        runner = SAETrainingRunner(cfg)
        sae = runner.run()
        saes_at_l1.append(sae)
        clear_gpu_memory()
    
    # Compute PWMCC between the two SAEs
    d1 = saes_at_l1[0].W_dec.data
    d2 = saes_at_l1[1].W_dec.data
    pwmcc = compute_pwmcc(d1, d2)
    
    # Estimate L0 (would need actual data for precise measurement)
    # For now, use L1 as proxy
    
    random_baseline = compute_random_baseline(d1.shape[1], d1.shape[0])
    ratio = pwmcc / random_baseline
    
    print(f'  PWMCC: {pwmcc:.4f} ({ratio:.2f}× random)')
    
    stability_results.append({
        'l1_coefficient': l1_coef,
        'pwmcc': pwmcc,
        'random_baseline': random_baseline,
        'ratio': ratio
    })
    
    # Clean up
    del saes_at_l1
    clear_gpu_memory()

print('\n✓ Experiment complete')

In [ ]:
#@title 11. Visualize Results
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Stability vs L1 coefficient
ax1 = axes[0]
l1s = [r['l1_coefficient'] for r in stability_results]
ratios = [r['ratio'] for r in stability_results]

ax1.plot(l1s, ratios, 'o-', markersize=10, linewidth=2, color='steelblue')
ax1.axhline(y=1.0, color='red', linestyle='--', label='Random baseline')
ax1.set_xlabel('L1 Coefficient (higher = sparser)', fontsize=12)
ax1.set_ylabel('Stability (× Random)', fontsize=12)
ax1.set_title('Stability vs Sparsity (LLM SAEs)', fontsize=14)
ax1.set_xscale('log')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: PWMCC values
ax2 = axes[1]
pwmccs = [r['pwmcc'] for r in stability_results]
baselines = [r['random_baseline'] for r in stability_results]

x = range(len(l1s))
ax2.bar(x, pwmccs, alpha=0.7, label='PWMCC', color='steelblue')
ax2.plot(x, baselines, 'r--', label='Random baseline', linewidth=2)
ax2.set_xlabel('L1 Coefficient', fontsize=12)
ax2.set_ylabel('PWMCC', fontsize=12)
ax2.set_title('Raw PWMCC Values', fontsize=14)
ax2.set_xticks(x)
ax2.set_xticklabels([f'{l1:.0e}' for l1 in l1s])
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('llm_sae_stability_results.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n✓ Figure saved to llm_sae_stability_results.png')

In [ ]:
#@title 12. Summary and Comparison with Algorithmic Tasks
print('=' * 70)
print('SUMMARY: LLM SAE Stability Analysis')
print('=' * 70)

print('\n📊 Results:')
for r in stability_results:
    print(f"  L1={r['l1_coefficient']:.0e}: PWMCC={r['pwmcc']:.4f} ({r['ratio']:.2f}× random)")

# Compute correlation
l1s = np.array([r['l1_coefficient'] for r in stability_results])
ratios = np.array([r['ratio'] for r in stability_results])

if len(l1s) > 2:
    corr = np.corrcoef(np.log(l1s), ratios)[0, 1]
    print(f'\n📈 Correlation (log L1 vs Stability): {corr:.3f}')
    
    if corr < -0.5:
        print('  → Higher sparsity = LOWER stability (matches algorithmic tasks!)')
    elif corr > 0.5:
        print('  → Higher sparsity = HIGHER stability (DIFFERENT from algorithmic tasks!)')
    else:
        print('  → Weak relationship')

print('\n' + '=' * 70)
print('COMPARISON WITH ALGORITHMIC TASKS')
print('=' * 70)
print('\nAlgorithmic Tasks (Modular Arithmetic):')
print('  - Stability DECREASES monotonically with L0')
print('  - Correlation: -0.725 (strong negative)')
print('  - Architecture-independent')

print('\nLLM SAEs (This Experiment):')
if len(stability_results) > 0:
    mean_ratio = np.mean(ratios)
    if mean_ratio < 1.2:
        print('  - Stability near random baseline')
    else:
        print(f'  - Stability {mean_ratio:.2f}× random baseline')
    
    if len(l1s) > 2 and corr < -0.3:
        print('  - Pattern MATCHES algorithmic tasks (higher sparsity = lower stability)')
    elif len(l1s) > 2 and corr > 0.3:
        print('  - Pattern DIFFERS from algorithmic tasks!')
    else:
        print('  - Inconclusive (need more data points)')

print('\n' + '=' * 70)

In [ ]:
#@title 13. Save Results
results = {
    'experiment': 'llm_sae_stability',
    'model': 'gemma-2-2b',
    'layer': LAYER,
    'stability_vs_sparsity': stability_results,
    'width_comparison': width_results if 'width_results' in dir() else [],
    'layer_comparison': layer_results if 'layer_results' in dir() else [],
}

with open('llm_sae_stability_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print('✓ Results saved to llm_sae_stability_results.json')

# Download files
from google.colab import files
files.download('llm_sae_stability_results.json')
files.download('llm_sae_stability_results.png')